In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
# from keras.layers.normalization import BatchNormalization
# from keras.optimizers import Adam
# from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
configpartno='85-EKA0190'

In [49]:
def readTrain():
    train =  pd.read_csv("./data/Parts_EQP_Output_ByMonth_20210407_van.csv")
    train= train[train['PART_NO']==configpartno]  
    train.drop(columns=['PART_NO','EQP_NO','MFG_MONTH','PM','TS','ENG','NST'],inplace=True)
    train.groupby(['STOCK_EVENT_TIME']).sum().reset_index()
    return train

In [50]:
df_train = readTrain()

,STOCK_EVENT_TIME,QTY
0,2015-01-31,0
1,2015-02-28,0
2,2015-03-31,0
3,2015-04-30,0
4,2015-05-31,0
...,...,...
70,2020-11-30,136
71,2020-12-31,156
72,2021-01-31,150
73,2021-02-28,117


In [51]:
def augFeatures(train):
  train["Date"] = pd.to_datetime(train["STOCK_EVENT_TIME"])
  train["year"] = train["Date"].dt.year
  train["month"] = train["Date"].dt.month
#   train["date"] = train["Date"].dt.day
#   train["day"] = train["Date"].dt.dayofweek
  return train

In [52]:
def normalize(train):
    train.drop(columns=["Date","STOCK_EVENT_TIME"], axis=1,inplace=True)
    # train = train.drop(["Date"], axis=1)
    train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train_norm

In [57]:
def buildTrain(train, pastDay=30, futureDay=5):
  X_train, Y_train = [], []
  for i in range(train.shape[0]-futureDay-pastDay):
    X_train.append(np.array(train.iloc[i:i+pastDay]))
    Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["QTY"]))
  return np.array(X_train), np.array(Y_train)

In [37]:
def shuffle(X,Y):
  np.random.seed(10)
  randomList = np.arange(X.shape[0])
  np.random.shuffle(randomList)
  return X[randomList], Y[randomList]

In [38]:

def splitData(X,Y,rate):
  X_train = X[int(X.shape[0]*rate):]
  Y_train = Y[int(Y.shape[0]*rate):]
  X_val = X[:int(X.shape[0]*rate)]
  Y_val = Y[:int(Y.shape[0]*rate)]
  return X_train, Y_train, X_val, Y_val

In [58]:
# read SPY.csv
train = readTrain()

# Augment the features (year, month, date, day)
train_Aug = augFeatures(train)

In [59]:
train_Aug

,STOCK_EVENT_TIME,QTY,Date,year,month
3150,2015-01-31,0,2015-01-31,2015,1
3151,2015-02-28,0,2015-02-28,2015,2
3152,2015-03-31,0,2015-03-31,2015,3
3153,2015-04-30,0,2015-04-30,2015,4
3154,2015-05-31,0,2015-05-31,2015,5
...,...,...,...,...,...
3745,2020-11-30,15,2020-11-30,2020,11
3746,2020-12-31,20,2020-12-31,2020,12
3747,2021-01-31,15,2021-01-31,2021,1
3748,2021-02-28,5,2021-02-28,2021,2


In [60]:
# Normalization
train_norm = normalize(train_Aug)

In [61]:

# build Data, use last 30 days to predict next 5 days
X_train, Y_train = buildTrain(train_norm, 30, 5)

# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X_train, Y_train)

# split training data and validation data
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)
# X_trian: (5710, 30, 10)
# Y_train: (5710, 5, 1)
# X_val: (634, 30, 10)
# Y_val: (634, 5, 1)

In [67]:
def buildOneToOneModel(shape):
    model = tf.keras.Sequential()
    model.add( tf.keras.layers.LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
    # output shape: (1, 1)
    model.add(  tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1)))    # or use model.add(Dense(1))
    model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam())
    model.summary()
    return model





    # model = tf.keras.Sequential([
    #         tf.keras.layers.Dense(units=32,activation = 'relu',input_shape=[X.shape[1]]),
    #         tf.keras.layers.Dense(units=16,activation = 'relu'),
    #         tf.keras.layers.Dense(units=4,activation = 'relu'), 
    #         tf.keras.layers.Dense(units=1)
    #         ]) 
    #     model.compile(loss='mean_squared_error', 
    #         optimizer=tf.keras.optimizers.Adam(0.001),
    #         metrics=[tf.keras.metrics.MeanAbsoluteError()]) 

    #     history = model.fit(X, y, epochs=500, batch_size=16, verbose=True,
    #         validation_split=0.01)   
    #     return model

In [69]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 1)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]

model = buildOneToOneModel(X_train.shape)
callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 1, 10)             560       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1, 1)              11        
Total params: 571
Trainable params: 571
Non-trainable params: 0
_________________________________________________________________
Train on 539 samples, validate on 59 samples
Epoch 1/1000
539/539 [==============================] - 4s 7ms/sample - loss: 0.0752 - val_loss: 0.0639
Epoch 2/1000
539/539 [==============================] - 0s 54us/sample - loss: 0.0740 - val_loss: 0.0631
Epoch 3/1000
539/539 [==============================] - 0s 61us/sample - loss: 0.0728 - val_loss: 0.0623
Epoch 4/1000
539/539 [==============================] - 0s 61us/sample - loss: 0.0718 - val_loss: 0.0615
Epoch 5/1000
539/539 [==============================] 

In [71]:
#多對一模型
def buildManyToOneModel(shape):
#   model = Sequential()
#   model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
#   # output shape: (1, 1)
#   model.add(Dense(1))
#   model.compile(loss="mse", optimizer="adam")
#   model.summary()
#   return model
    model = tf.keras.Sequential([
            tf.keras.layers.LSTM(units=10, input_length=shape[1], input_dim=shape[2]),
            tf.keras.layers.Dense(units=1),
            ]) 
    model.compile(loss='mean_squared_error', 
            optimizer=tf.keras.optimizers.Adam(),
            # metrics=[tf.keras.metrics.MeanAbsoluteError()]
            ) 
    model.summary()
    return model

In [73]:
train_norm

,QTY,year,month
3150,-0.268022,-0.440000,-0.483636
3151,-0.268022,-0.440000,-0.392727
3152,-0.268022,-0.440000,-0.301818
3153,-0.268022,-0.440000,-0.210909
3154,-0.268022,-0.440000,-0.120000
...,...,...,...
3745,-0.068022,0.393333,0.425455
3746,-0.001356,0.393333,0.516364
3747,-0.068022,0.560000,-0.483636
3748,-0.201356,0.560000,-0.392727


In [75]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 30, 1)
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

model = buildManyToOneModel(X_train.shape)
callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 10)                560       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 571
Trainable params: 571
Non-trainable params: 0
_________________________________________________________________
Train on 513 samples, validate on 56 samples
Epoch 1/1000
513/513 [==============================] - 4s 7ms/sample - loss: 0.0555 - val_loss: 0.0704
Epoch 2/1000
513/513 [==============================] - 0s 146us/sample - loss: 0.0502 - val_loss: 0.0640
Epoch 3/1000
513/513 [==============================] - 0s 154us/sample - loss: 0.0462 - val_loss: 0.0589
Epoch 4/1000
513/513 [==============================] - 0s 154us/sample - loss: 0.0424 - val_loss: 0.0545
Epoch 5/1000
513/513 [=============================

In [78]:
def buildOneToManyModel(shape):
#   model = Sequential()
#   model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
#   # output shape: (5, 1)
#   model.add(Dense(1))
#   model.add(RepeatVector(5))
#   model.compile(loss="mse", optimizer="adam")
#   model.summary()
#   return model
    model = tf.keras.Sequential([
            tf.keras.layers.LSTM(units=10, input_length=shape[1], input_dim=shape[2]),
            tf.keras.layers.Dense(units=1),
            tf.keras.layers.RepeatVector(5),
            ]) 
    model.compile(loss='mean_squared_error', 
            optimizer=tf.keras.optimizers.Adam(),
            # metrics=[tf.keras.metrics.MeanAbsoluteError()]
            ) 
    model.summary()
    return model

In [79]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildOneToManyModel(X_train.shape)
callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

/1000
535/535 [==============================] - 0s 69us/sample - loss: 0.0243 - val_loss: 0.0190
Epoch 44/1000
535/535 [==============================] - 0s 88us/sample - loss: 0.0239 - val_loss: 0.0186
Epoch 45/1000
535/535 [==============================] - 0s 56us/sample - loss: 0.0236 - val_loss: 0.0183
Epoch 46/1000
535/535 [==============================] - 0s 62us/sample - loss: 0.0232 - val_loss: 0.0180
Epoch 47/1000
535/535 [==============================] - 0s 84us/sample - loss: 0.0228 - val_loss: 0.0177
Epoch 48/1000
535/535 [==============================] - 0s 105us/sample - loss: 0.0225 - val_loss: 0.0174
Epoch 49/1000
535/535 [==============================] - 0s 65us/sample - loss: 0.0222 - val_loss: 0.0172
Epoch 50/1000
535/535 [==============================] - 0s 95us/sample - loss: 0.0220 - val_loss: 0.0170
Epoch 51/1000
535/535 [==============================] - 0s 67us/sample - loss: 0.0217 - val_loss: 0.0167
Epoch 52/1000
535/535 [==============================

# 多對多模型 (輸入與輸出相同長度)
將return_sequences 設為True ，再用TimeDistributed(Dense(1)) 將輸出調整為(5,1)

In [82]:
def buildManyToManyModel(shape):
#   model = Sequential()
#   model.add(LSTM(10, input_length=shape[1], input_dim=shape[2], return_sequences=True))
#   # output shape: (5, 1)
#   model.add(TimeDistributed(Dense(1)))
#   model.compile(loss="mse", optimizer="adam")
#   model.summary()
#   return model
    model = tf.keras.Sequential([
            tf.keras.layers.LSTM(units=10, input_length=shape[1], input_dim=shape[2], return_sequences=True),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1)),
            ]) 
    model.compile(loss='mean_squared_error', 
            optimizer=tf.keras.optimizers.Adam(),
            # metrics=[tf.keras.metrics.MeanAbsoluteError()]
            ) 
    model.summary()
    return model

In [83]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 5, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildManyToManyModel(X_train.shape)
callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=10, verbose=0, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


ple - loss: 0.0189 - val_loss: 0.0175
Epoch 540/1000
531/531 [==============================] - 0s 98us/sample - loss: 0.0189 - val_loss: 0.0174
Epoch 541/1000
531/531 [==============================] - 0s 85us/sample - loss: 0.0189 - val_loss: 0.0174
Epoch 542/1000
531/531 [==============================] - 0s 105us/sample - loss: 0.0189 - val_loss: 0.0176
Epoch 543/1000
531/531 [==============================] - 0s 81us/sample - loss: 0.0189 - val_loss: 0.0177
Epoch 544/1000
531/531 [==============================] - 0s 77us/sample - loss: 0.0188 - val_loss: 0.0177
Epoch 545/1000
531/531 [==============================] - 0s 113us/sample - loss: 0.0188 - val_loss: 0.0176
Epoch 546/1000
531/531 [==============================] - 0s 79us/sample - loss: 0.0188 - val_loss: 0.0175
Epoch 547/1000
531/531 [==============================] - 0s 79us/sample - loss: 0.0188 - val_loss: 0.0175
Epoch 548/1000
531/531 [==============================] - 0s 98us/sample - loss: 0.0188 - val_loss: 0.01